# 什么是PyTorch？

PyTorch是一个基于python的科学计算包，主要针对两类人群：

* 作为NumPy的替代品，可以利用GPU的性能进行计算
* 作为一个高灵活性、速度快的深度学习平台

## 1 入门

### 1.1 张量

`Tensor`（张量）类似于`NumPy`的`ndarray`，但还可以在GPU上使用来加速计算。

In [1]:
from __future__ import print_function
import torch

创建一个没有初始化的5*3矩阵：

In [2]:
x = torch.empty(5, 3)
print(x)

tensor([[5.7858e-39, 8.4490e-39, 5.3266e-39],
        [7.8061e-39, 9.2755e-39, 1.0561e-38],
        [1.0010e-38, 5.9694e-39, 8.9082e-39],
        [1.0194e-38, 9.1837e-39, 4.6837e-39],
        [9.9184e-39, 9.0000e-39, 1.0745e-38]])


创建一个随机初始化矩阵：

In [3]:
x = torch.rand(5, 3)
print(x)

tensor([[0.2337, 0.7502, 0.6225],
        [0.7343, 0.9717, 0.3049],
        [0.7759, 0.1603, 0.9447],
        [0.6374, 0.8291, 0.2985],
        [0.9360, 0.3643, 0.4708]])


构造一个填满0且数据类型为`long`的矩阵:

In [4]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


直接从数据构造张量：

In [5]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


或者根据已有的tensor建立新的tensor。除非用户提供新的值，否则这些方法将重用输入张量的属性，例如dtype等：

In [6]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)                                      # result has the same size

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-0.8644,  0.9352,  1.1714],
        [ 0.4736, -0.6051, -1.7749],
        [-0.5351, -1.3570,  1.3326],
        [ 1.1750, -0.7485, -0.9362],
        [-0.9991,  0.5784,  0.7740]])


获取它的形状：

In [7]:
print(x.size())

torch.Size([5, 3])


> **注意**：
>
> `torch.Size`本质上还是`tuple`，所以支持tuple的一切操作。

### 1.2 运算

一种运算有多种语法。在下面的示例中，我们将研究加法运算。

加法：形式一

In [8]:
y = torch.rand(5, 3)
print(x + y)

tensor([[-0.6116,  1.3618,  1.8453],
        [ 0.9878,  0.3476, -1.5394],
        [-0.1031, -0.6953,  1.5703],
        [ 2.0613, -0.4191, -0.7209],
        [-0.4310,  0.8837,  0.8443]])


加法：形式二

In [9]:
print(torch.add(x, y))

tensor([[-0.6116,  1.3618,  1.8453],
        [ 0.9878,  0.3476, -1.5394],
        [-0.1031, -0.6953,  1.5703],
        [ 2.0613, -0.4191, -0.7209],
        [-0.4310,  0.8837,  0.8443]])


加法：给定一个输出张量作为参数

In [10]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[-0.6116,  1.3618,  1.8453],
        [ 0.9878,  0.3476, -1.5394],
        [-0.1031, -0.6953,  1.5703],
        [ 2.0613, -0.4191, -0.7209],
        [-0.4310,  0.8837,  0.8443]])


加法：原位/原地操作（in-place）

In [11]:
# adds x to y
y.add_(x)
print(y)

tensor([[-0.6116,  1.3618,  1.8453],
        [ 0.9878,  0.3476, -1.5394],
        [-0.1031, -0.6953,  1.5703],
        [ 2.0613, -0.4191, -0.7209],
        [-0.4310,  0.8837,  0.8443]])


>注意：
>
>任何一个in-place改变张量的操作后面都固定一个`_`。例如`x.copy_(y)`、`x.t_()`将更改x


也可以使用像标准的NumPy一样的各种索引操作：

In [12]:
print(x[:, 1])

tensor([ 0.9352, -0.6051, -1.3570, -0.7485,  0.5784])


改变形状：如果想改变形状，可以使用`torch.view`

In [13]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


如果是仅包含一个元素的tensor，可以使用`.item()`来得到对应的python数值

In [14]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([0.9748])
0.9747663140296936


>后续阅读：
>
>超过100中tensor的运算操作，包括转置，索引，切片，数学运算，
线性代数，随机数等，具体访问[这里](https://pytorch.org/docs/stable/torch.html)

## 2 NumPy桥

将一个Torch张量转换为一个NumPy数组是轻而易举的事情，反之亦然。

Torch张量和NumPy数组将共享它们的底层内存位置，更改一个将更改另一个。


### 2.1 将torch的Tensor转化为NumPy数组

In [15]:
a = torch.ones(5)
print(a)

tensor([1., 1., 1., 1., 1.])


In [16]:
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


看NumPy数组是如何改变里面的值的：

a.add_(1)
print(a)
print(b)

### 2.2 将NumPy数组转化为Torch张量

看改变NumPy数组是如何自动改变Torch张量的：

In [17]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


CPU上的所有张量(CharTensor除外)都支持转换为NumPy以及由NumPy转换回来。

## 3 CUDA上的张量

张量可以使用`.to`方法移动到任何设备（device）上：

In [18]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([1.9748], device='cuda:0')
tensor([1.9748], dtype=torch.float64)
